<a href="https://colab.research.google.com/github/BoMacArthur/WRIA_1_Irrigation_Models/blob/main/Google_Colab_Earth_Engine_Scripts/Optuna_Optimization_NINA_ET_No_Outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install optuna

In [ ]:
import ee
import geemap
import optuna
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pickle
from google.colab import output
import pandas as pd

In [ ]:
cloud_project = 'ee-bomacarthur'

try:
  ee.Initialize(project=cloud_project)
except:
  ee.Authenticate()
  ee.Initialize(project=cloud_project)


In [ ]:
m = geemap.Map(width=800)
m

In [ ]:
# Bring in the water balance model from Google Earth Engine
# Load datasets
fields = ee.FeatureCollection("projects/ee-bomacarthur/assets/Test_Fields/optimizationFields")
all_fields = ee.FeatureCollection("projects/ee-bomacarthur/assets/zonalStats/allFieldsZonalStatsEnsem")
etAndPrcpCol = ee.ImageCollection("projects/ee-bomacarthur/assets/etAndPrcp")
geometry = ee.Geometry.Polygon(
    [[
        [-122.33689842612226, 48.72511972728958],
        [-122.33689842612226, 48.70971669597075],
        [-122.31887398154218, 48.70971669597075],
        [-122.31887398154218, 48.72511972728958],
        [-122.33689842612226, 48.72511972728958]  # Close the polygon
    ]]
)

# Sort the et and prcp collection by date
et_and_prcp = etAndPrcpCol.sort('system:time_start')

# Count number of fields
size = fields.size()

def NINA_ET_model(irr_constant):
    # Calculate ET Zonal Stats
    def map_et(image):
        date = ee.Date(image.get('date'))
        filter_fields = fields.filterDate(date, date.advance(1, 'month'))
        property_names = fields.first().propertyNames().cat(['et'])
        zonal_stats = image.select(['ETa']).reduceRegions(
            collection=filter_fields,
            reducer=ee.Reducer.mean(),
            scale=30,
            crs='EPSG:32610',
            tileScale=16
        )
        return zonal_stats.map(lambda f: f.set('et', f.get('mean')).select(property_names))

    test_fields = et_and_prcp.map(map_et).flatten()

    # Filter by geometry
    test_fields_filtered = test_fields.filter(ee.Filter.bounds(geometry).Not())
    test_fields_sv = test_fields.filter(ee.Filter.bounds(geometry))
    all_fields_filtered = all_fields.filter(ee.Filter.eq('Irrigation', 'None'))

    # Function to find nearest fields and calculate ETi
    def map_fields(test_feature, buffer_dist):
        buffer = test_feature.geometry().buffer(buffer_dist)
        start = ee.String(test_feature.get('startDate'))
        candidates = all_fields_filtered \
            .filterBounds(buffer) \
            .filter(ee.Filter.eq('DateStart', start))

        def enrich(nir_feature):
            distance = test_feature.geometry().distance(nir_feature.geometry())
            return nir_feature.set({
                'distance': distance,
                'et': nir_feature.get('et'),
                'system:time_start': ee.Date(start).millis(),
                'system:time_end': ee.Date(start).advance(1, 'month').millis()
            })

        nearest = candidates.map(enrich)
        nearest_filtered = nearest.filter(ee.Filter.gt('distance', 100))
        et_irrigated = ee.Number(test_feature.get('et'))
        et_non_irrigated_avg = ee.Number(nearest_filtered.aggregate_mean('et'))
        adjusted_et = et_irrigated.subtract(et_non_irrigated_avg)

        return test_feature.set({
            'closestFieldsEtAvg': et_non_irrigated_avg,
            'numberOfClosestFields': nearest_filtered.size(),
            'ETi': adjusted_et
        })

    test_fields_mapped = test_fields_filtered.map(lambda f: map_fields(f, 1000))
    test_fields_mapped_sv = test_fields_sv.map(lambda f: map_fields(f, 7000))

    # Merge and process ETi
    positive_et = ee.FeatureCollection([
        test_fields_mapped.filter(ee.Filter.gt('ETi', 0)),
        test_fields_mapped_sv.filter(ee.Filter.gt('ETi', 0))
    ]).flatten()

    negative_et = ee.FeatureCollection([
        test_fields_mapped.filter(ee.Filter.lte('ETi', 0)),
        test_fields_mapped_sv.filter(ee.Filter.lte('ETi', 0))
    ]).flatten()

    # Define conversion functions
    def model_volume_positive(feature):
        area = ee.Number(feature.get('area'))
        et = ee.Number(feature.get('ETi')).abs()
        water_meter = ee.Number(feature.get('waterMeterVolume'))
        irrigation_eff = irr_constant
        return feature.set({
            'waterModelVolume': et.multiply(area).divide(1000).divide(irrigation_eff),
            'ETi': et,
            'waterModelDepth': et.divide(irrigation_eff),
            'waterMeterDepth': water_meter.divide(area).multiply(1000)
        })

    def model_volume_negative(feature):
        area = ee.Number(feature.get('area'))
        water_meter = ee.Number(feature.get('waterMeterVolume'))
        return feature.set({
            'waterModelVolume': 0,
            'ETi': 0,
            'waterModelDepth': 0,
            'waterMeterDepth': water_meter.divide(area).multiply(1000)
        })

    positive_vol = positive_et.map(model_volume_positive)
    negative_vol = negative_et.map(model_volume_negative)

    all_results = ee.FeatureCollection([positive_vol, negative_vol]).flatten().sort('system:time_start')

    # Add error columns
    def calc_error(feature):
        wm = ee.Number(feature.get('waterMeterVolume'))
        mo = ee.Number(feature.get('waterModelVolume'))
        wmd = ee.Number(feature.get('waterMeterDepth'))
        mod = ee.Number(feature.get('waterModelDepth'))
        return feature.set({
            'meterMinusModelVolume': wm.subtract(mo),
            'meterMinusModelVolumeAbs': wm.subtract(mo).abs(),
            'meterMinusModelDepth': wmd.subtract(mod),
            'meterMinusModelDepthAbs': wmd.subtract(mod).abs()
        })

    test_results = all_results.map(calc_error)

    # MAE calculations
    size = test_fields.size()

    mae_depth = ee.Number(
        test_results.reduceColumns(
            reducer=ee.Reducer.sum(),
            selectors=['meterMinusModelDepthAbs']
        ).get('sum')
    ).divide(size)

    return mae_depth.getInfo()




In [ ]:
# Create a loop to run the model 101 times with Irrigation efficiency values from 0 to 1 in 0.01 steps
results = []

for i in range(101):
    irr_efficiency = i / 100  # Steps of 0.01 from 0.00 to 1.00
    try:
        mae = NINA_ET_model(irr_efficiency)
        results.append({'irr_efficiency': irr_efficiency, 'mae': mae})
        print(f"irr_constant, {irr_efficiency:.2f}, MAE, {mae:.6f}")
    except Exception as e:
        print(f"Error at irr_efficiency={irr_efficiency:.2f}: {e}")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Convert results to data frame and save to google drive as csv
df = pd.DataFrame(results)
csv_path = '/content/drive/My Drive/NINA_ET_test_results/optuna_optimization_NINA_ET.csv'
df.to_csv(csv_path, index=False)